# DS 2002 Midterm Project Script: Adventureworks Purchase Orders

## Part 1: SQL 

In [1]:
# import packages

import os
import numpy
import pandas as pd
import datetime
from sqlalchemy import create_engine

In [2]:
# initialize connection variables

host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "Ronddej%40mbe2003"

src_dbname = "adventureworks"
dst_dbname = "adventureworks_dw"

In [3]:
# define sql dataframe functions 

def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection)
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [4]:
# define connection string and engine

conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

### create adventureworks_dw

In [5]:
# drop dw if it exists, create and then use

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

### date dimension 

Based on intial exploration of the Adventureworks data, I discovered that it had dates as far back as 1998, so I edited the "Lab_02c_Create_Populate_Dim_Date.sql" file to have a start date of 01-01-1995, and then I ran the file in MySQL to create the date dimension. 

### employee dimension

In [12]:
# get employee data from source db 

sql_employees = "SELECT * FROM adventureworks.employee;"
df_employees = get_dataframe(user_id, pwd, host_name, src_dbname, sql_employees)
df_employees.head(5)

,EmployeeID,NationalIDNumber,ContactID,LoginID,ManagerID,Title,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,14417807,1209,adventure-works\guy1,16.0,Production Technician - WC60,1972-05-15,M,M,1996-07-31,b'\x00',21,30,b'\x01',b'J\xd0\xe1\xaa7\xc2tI\xb4\xd5\x93RGsw\x18',2004-07-31
1,2,253022876,1030,adventure-works\kevin0,6.0,Marketing Assistant,1977-06-03,S,M,1997-02-26,b'\x00',42,41,b'\x01',b'@\x02H\x1b\xc0\x95\x0fA\xa7\x17\xeb)\x94<\x8...,2004-07-31
2,3,509647174,1002,adventure-works\roberto0,12.0,Engineering Manager,1964-12-13,M,M,1997-12-12,b'\x01',2,21,b'\x01',"b',\xfb\xbb\x9b\xbb\xef\x17B\x9a\xb7\xf9v\x892...",2004-07-31
3,4,112457891,1290,adventure-works\rob0,3.0,Senior Tool Designer,1965-01-23,S,M,1998-01-05,b'\x00',48,80,b'\x01',b'UytY\xb8\x87?D\x8e\xd4\xf8\xad:\xfd\xf3\xa9',2004-07-31
4,5,480168528,1009,adventure-works\thierry0,263.0,Tool Designer,1949-08-29,M,M,1998-01-11,b'\x00',9,24,b'\x01',b'qQ\x95\x1ds\xe7\xadO\x83\x82@\xfd\x89\x8d]M',2004-07-31


In [13]:
# clean up SalariedFlag and Current Flag 

df_employees['SalariedFlag'] = df_employees['SalariedFlag'].str[-1]
df_employees['CurrentFlag'] = df_employees['CurrentFlag'].str[-1]

In [14]:
# get contact data from source db 

sql_contacts = "SELECT * FROM adventureworks.contact;"
df_contacts = get_dataframe(user_id, pwd, host_name, src_dbname, sql_contacts)
df_contacts.head(5)

,ContactID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailAddress,EmailPromotion,Phone,PasswordHash,PasswordSalt,AdditionalContactInfo,rowguid,ModifiedDate
0,1,b'\x00',Mr.,Gustavo,None,Achong,None,gustavo0@adventure-works.com,2,398-555-0132,F295DC2A87C7FD93151A2BC232BEAFF1133F95A2,TVGHbhY=,"<AdditionalContactInfo xmlns=""http://schemas.m...",b'\xd32\xc1\xd4\xb5\xfc1B\x9d\xd5\x88\x8aT\xbe...,2005-05-16 16:33:33
1,2,b'\x00',Ms.,Catherine,R.,Abel,None,catherine0@adventure-works.com,1,747-555-0171,19712A42FC40F14655499D5058E6A877FE9C367B,rpyd5Tw=,"<AdditionalContactInfo xmlns=""http://schemas.m...","b'R\x05N\xd5&\xc2""L\xaf;v,\xa8T\xcd\xd3'",2005-05-16 16:33:33
2,3,b'\x00',Ms.,Kim,None,Abercrombie,None,kim2@adventure-works.com,0,334-555-0137,CBCD65769648CCBA7693450991E4388894F13723,rrgbG/U=,"<AdditionalContactInfo xmlns=""http://schemas.m...",b'H\xdb\xcb\xf7D\x0b\x0eG\x9f7p`Do\xbf\xb9',2005-05-16 16:33:33
3,4,b'\x00',Sr.,Humberto,None,Acevedo,None,humberto0@adventure-works.com,2,599-555-0127,69CF91B5628FA4217743B30C5FE14B5B80A03DB2,F5qyyxs=,"<AdditionalContactInfo xmlns=""http://schemas.m...",b'6\xd3AZ\xcf\x84\xd7D\xb1+\x83\xb6KQ\x1f~',2005-05-16 16:33:33
4,5,b'\x00',Sra.,Pilar,None,Ackerman,None,pilar1@adventure-works.com,0,1 (11) 500 555-0132,F57E03FEA2FD0F74684C20758110CC7860F67523,/RPjvXw=,"<AdditionalContactInfo xmlns=""http://schemas.m...",b'\xab\xb8\x1f\xdf##0C\x9a\xb8T\xe1<\xe6\xd8\xf9',2005-05-16 16:33:33


In [15]:
# drop irrelevant columns from df_contacts

columns = ['NameStyle','Title','MiddleName','Suffix','EmailPromotion','PasswordHash','PasswordSalt','AdditionalContactInfo', 'rowguid',
       'ModifiedDate']
df_contacts.drop(columns, axis=1, inplace=True)

In [16]:
# merge employees and contacts on contactid

df_employees = pd.merge(df_employees, df_contacts, on='ContactID', how='inner')
df_employees.drop(['ContactID'], axis=1, inplace=True)

In [17]:
# drop irrelevant columns from df_employees

columns = ['NationalIDNumber','LoginID','ManagerID','BirthDate','rowguid']
df_employees.drop(columns, axis=1, inplace=True)

In [18]:
# lookup HireDate and ModifiedDate as hire_date_key and modified_date_key
sql_dim_date = "SELECT date_key, full_date FROM adventureworks_dw.dim_date;"
df_dim_date = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_dim_date)

df_dim_hire_date = df_dim_date.rename(columns={"date_key" : "hire_date_key", "full_date" : "HireDate"})
df_employees.HireDate = df_employees.HireDate.astype('datetime64[ns]').dt.date
df_employees = pd.merge(df_employees, df_dim_hire_date, on='HireDate', how='left')
df_employees.drop(columns=['HireDate'], inplace=True)

df_dim_modified_date = df_dim_date.rename(columns={"date_key" : "modified_date_key", "full_date" : "ModifiedDate"})
df_employees.ModifiedDate = df_employees.ModifiedDate.astype('datetime64[ns]').dt.date
df_employees = pd.merge(df_employees, df_dim_modified_date, on='ModifiedDate', how='left')
df_employees.drop(columns=['ModifiedDate'], inplace=True)
df_employees.head(2)

,EmployeeID,Title,MaritalStatus,Gender,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,FirstName,LastName,EmailAddress,Phone,hire_date_key,modified_date_key
0,1,Production Technician - WC60,M,M,0,21,30,1,Guy,Gilbert,guy1@adventure-works.com,320-555-0195,19960731,20040731
1,2,Marketing Assistant,S,M,0,42,41,1,Kevin,Brown,kevin0@adventure-works.com,150-555-0189,19970226,20040731


In [19]:
# reorder columns 
order_col = ['EmployeeID','FirstName','LastName','Title','EmailAddress','Phone','MaritalStatus','Gender','hire_date_key',
    'SalariedFlag','VacationHours','SickLeaveHours','CurrentFlag','modified_date_key']
df_employees = df_employees[order_col]

In [20]:
# insert primary key 

df_employees.insert(0, "employee_key", range(1, df_employees.shape[0]+1))

In [21]:
# check work 

df_employees.head(5)

,employee_key,EmployeeID,FirstName,LastName,Title,EmailAddress,Phone,MaritalStatus,Gender,hire_date_key,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,modified_date_key
0,1,1,Guy,Gilbert,Production Technician - WC60,guy1@adventure-works.com,320-555-0195,M,M,19960731,0,21,30,1,20040731
1,2,2,Kevin,Brown,Marketing Assistant,kevin0@adventure-works.com,150-555-0189,S,M,19970226,0,42,41,1,20040731
2,3,3,Roberto,Tamburello,Engineering Manager,roberto0@adventure-works.com,212-555-0187,M,M,19971212,1,2,21,1,20040731
3,4,4,Rob,Walters,Senior Tool Designer,rob0@adventure-works.com,612-555-0100,S,M,19980105,0,48,80,1,20040731
4,5,5,Thierry,D'Hers,Tool Designer,thierry0@adventure-works.com,168-555-0183,M,M,19980111,0,9,24,1,20040731


In [22]:
# push to SQL

table_name = "dim_employees"
primary_key = "employee_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_employees, table_name, primary_key, db_operation)

### product dimension

In [23]:
# get product data from source db 

sql_products = "SELECT * FROM adventureworks.product;"
df_products = get_dataframe(user_id, pwd, host_name, src_dbname, sql_products)
df_products.head(5)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,b'\x00',b'\x00',None,1000,750,0.0,0.0,...,None,None,None,NaN,NaN,1998-06-01,NaT,None,b'\xb7\x15Bi\xf7\x08\rL\xac\xb1\xd74\xbaD\xc0\...,2004-03-11 10:01:36
1,2,Bearing Ball,BA-8327,b'\x00',b'\x00',None,1000,750,0.0,0.0,...,None,None,None,NaN,NaN,1998-06-01,NaT,None,b' <\xaeX:OIG\xa7\xd4\xd5h\x80l\xc57',2004-03-11 10:01:36
2,3,BB Ball Bearing,BE-2349,b'\x01',b'\x00',None,800,600,0.0,0.0,...,None,None,None,NaN,NaN,1998-06-01,NaT,None,b'\xd2\xae!\x9c\xfa[\x18O\xbc\xb8\xf1\x168\xdc.N',2004-03-11 10:01:36
3,4,Headset Ball Bearings,BE-2908,b'\x00',b'\x00',None,800,600,0.0,0.0,...,None,None,None,NaN,NaN,1998-06-01,NaT,None,b'\xcb\xd6\xfe\xec\xffQ\xb5I\xb0l}\x8a\xc84\xd...,2004-03-11 10:01:36
4,316,Blade,BL-2036,b'\x01',b'\x00',None,800,600,0.0,0.0,...,None,None,None,NaN,NaN,1998-06-01,NaT,None,b'P\x97>\xe7;`1A\x89\xf5=\xd1^\xd5\xff\x80',2004-03-11 10:01:36


In [24]:
# clean up MakeFlag and FinishedGoodsFlag 

df_products['MakeFlag'] = df_products['MakeFlag'].str[-1]
df_products['FinishedGoodsFlag'] = df_products['FinishedGoodsFlag'].str[-1]

In [25]:
# drop irrelevant columns from df_products (columns that have a lot of nulls, etc)

columns = ['Color', 'Size', 'SizeUnitMeasureCode', 'WeightUnitMeasureCode', 'Weight', 'ProductLine', 'Class', 'Style', 
           'ProductSubcategoryID', 'ProductModelID', 'SellStartDate', 'SellEndDate', 'DiscontinuedDate', 'rowguid']
df_products.drop(columns, axis=1, inplace=True)

In [26]:
# lookup ModifiedDate as modified_date_key

df_products.ModifiedDate = df_products.ModifiedDate.astype('datetime64[ns]').dt.date
df_products = pd.merge(df_products, df_dim_modified_date, on='ModifiedDate', how='left')
df_products.drop(columns=['ModifiedDate'], inplace=True)

In [27]:
# insert primary key 

df_products.insert(0, "product_key", range(1, df_products.shape[0]+1))

In [28]:
# check work 

df_products.head(5)

,product_key,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,DaysToManufacture,modified_date_key
0,1,1,Adjustable Race,AR-5381,0,0,1000,750,0.0,0.0,0,20040311
1,2,2,Bearing Ball,BA-8327,0,0,1000,750,0.0,0.0,0,20040311
2,3,3,BB Ball Bearing,BE-2349,1,0,800,600,0.0,0.0,1,20040311
3,4,4,Headset Ball Bearings,BE-2908,0,0,800,600,0.0,0.0,0,20040311
4,5,316,Blade,BL-2036,1,0,800,600,0.0,0.0,1,20040311


In [29]:
# push to SQL

table_name = "dim_products"
primary_key = "product_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_products, table_name, primary_key, db_operation)

## Part 2: local file

### vendor dimension

In [30]:
# import csv file to a dataframe

data_dir = os.path.join(os.getcwd())
data_file = os.path.join(data_dir, 'adventureworks_vendor.csv')

df_vendors = pd.read_csv(data_file, header=0)
df_vendors.head()

,VendorID,AccountNumber,Name,CreditRating,PreferredVendorStatus,ActiveFlag,PurchasingWebServiceURL,ModifiedDate
0,1,INTERNAT0001,International,1,1,1,NaN,2002-02-25 00:00:00
1,2,ELECTRON0002,Electronic Bike Repair & Supplies,1,1,1,NaN,2002-02-17 00:00:00
2,3,PREMIER0001,"Premier Sport, Inc.",1,1,1,NaN,2002-03-05 00:00:00
3,4,COMFORT0001,Comfort Road Bicycles,1,1,1,NaN,2002-01-24 00:00:00
4,5,METROSP0001,Metro Sport Equipment,1,1,1,NaN,2002-03-01 00:00:00


In [31]:
# drop irrelevant columns from df_vendors

columns = ['PurchasingWebServiceURL']
df_vendors.drop(columns, axis=1, inplace=True)

In [32]:
# lookup ModifiedDate as modified_date_key

df_vendors.ModifiedDate = df_vendors.ModifiedDate.astype('datetime64[ns]').dt.date
df_vendors = pd.merge(df_vendors, df_dim_modified_date, on='ModifiedDate', how='left')
df_vendors.drop(columns=['ModifiedDate'], inplace=True)

In [33]:
# insert primary key 

df_vendors.insert(0, "vendor_key", range(1, df_vendors.shape[0]+1))

In [34]:
# check work 

df_vendors.head()

,vendor_key,VendorID,AccountNumber,Name,CreditRating,PreferredVendorStatus,ActiveFlag,modified_date_key
0,1,1,INTERNAT0001,International,1,1,1,20020225
1,2,2,ELECTRON0002,Electronic Bike Repair & Supplies,1,1,1,20020217
2,3,3,PREMIER0001,"Premier Sport, Inc.",1,1,1,20020305
3,4,4,COMFORT0001,Comfort Road Bicycles,1,1,1,20020124
4,5,5,METROSP0001,Metro Sport Equipment,1,1,1,20020301


In [35]:
# push to SQl

table_name = "dim_vendors"
primary_key = "vendor_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_vendors, table_name, primary_key, db_operation)

## Part 3: mongoDB

In [36]:
# import packages

import json
import certifi
import pymongo
import sqlalchemy

In [37]:
# initialize connection variables

mysql_uid = "root"
mysql_pwd = "Ronddej%40mbe2003"
mysql_hostname = "localhost"

atlas_cluster_name = "cluster0.zrzzkeo"
atlas_user_name = "rvt9bx"
atlas_password = "Ronddejambe2003"

conn_str = {"local" : f"mongodb://localhost:27017/",
    "atlas" : f"mongodb+srv://{atlas_user_name}:{atlas_password}@{atlas_cluster_name}.mongodb.net"
}

src_dbname = "adventureworks"
dst_dbname = "adventureworks_dw"

In [38]:
# define sql and mongo db dataframe functions

def get_sql_dataframe(user_id, pwd, host_name, db_name, sql_query):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    conn = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, conn);
    conn.close()
    
    return dframe


def get_mongo_dataframe(connect_str, db_name, collection, query):
    '''Create a connection to MongoDB'''
    client = pymongo.MongoClient(connect_str)
    
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [39]:
# populate mongo db with json data (only run once)

client = pymongo.MongoClient(conn_str["atlas"], tlsCAFile=certifi.where())
db = client[src_dbname]

# Gets the path of the Current Working Directory for this Notebook
data_dir = os.path.join(os.getcwd())

json_files = {"shipmethod" : 'adventureworks_shipmethod.json'
             }

for file in json_files:
    db.drop_collection(file)
    json_file = os.path.join(data_dir, json_files[file])
    with open(json_file, 'r') as openfile:
        json_object = json.load(openfile)
        file = db[file]
        result = file.insert_many(json_object)

        
client.close()        

### ship method dimension

In [40]:
# get shipmethod data from mongo db 

query = {} # Select all elements (columns), and all documents (rows).
collection = "shipmethod"

df_shipmethod = get_mongo_dataframe(conn_str['atlas'], src_dbname, collection, query)
df_shipmethod.head(2)

,ShipMethodID,Name,ShipBase,ShipRate,rowguid,ModifiedDate
0,1,XRQ - TRUCK GROUND,3.95,0.99,None,1998-06-01 00:00:00
1,2,ZY - EXPRESS,9.95,1.99,None,1998-06-01 00:00:00


In [41]:
# drop irrelevant columns from df_shipmethod

columns = ['rowguid']
df_shipmethod.drop(columns, axis=1, inplace=True)

In [42]:
# lookup ModifiedDate as modified_date_key

df_shipmethod.ModifiedDate = df_shipmethod.ModifiedDate.astype('datetime64[ns]').dt.date
df_shipmethod = pd.merge(df_shipmethod, df_dim_modified_date, on='ModifiedDate', how='left')
df_shipmethod.drop(columns=['ModifiedDate'], inplace=True)

In [43]:
# insert primary key 

df_shipmethod.insert(0, "shipmethod_key", range(1, df_shipmethod.shape[0]+1))

In [44]:
# check work 

df_shipmethod.head()

,shipmethod_key,ShipMethodID,Name,ShipBase,ShipRate,modified_date_key
0,1,1,XRQ - TRUCK GROUND,3.95,0.99,19980601
1,2,2,ZY - EXPRESS,9.95,1.99,19980601
2,3,3,OVERSEAS - DELUXE,29.95,2.99,19980601
3,4,4,OVERNIGHT J-FAST,21.95,1.29,19980601
4,5,5,CARGO TRANSPORT 5,8.99,1.49,19980601


In [45]:
# push to SQl

table_name = "dim_shipmethod"
primary_key = "shipmethod_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_shipmethod, table_name, primary_key, db_operation)

## Part 4: fact table

### purchase orders fact table

In [46]:
# import purchase order header from source db 

purchase_order_header = "SELECT * FROM adventureworks.purchaseorderheader;"
df_fact_po = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, purchase_order_header)
df_fact_po.head(2)

,PurchaseOrderID,RevisionNumber,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate
0,1,0,4,244,83,3,2001-05-17,2001-05-26,201.0400,16.0832,5.0260,222.1492,2001-05-26
1,2,0,1,231,32,5,2001-05-17,2001-05-26,272.1015,21.7681,6.8025,300.6721,2001-05-26


### import surrogate and primary keys for each dimension

In [47]:
# dim employees

sql_dim_employees = "SELECT employee_key, EmployeeID FROM adventureworks_dw.dim_employees;"
df_dim_employees = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_dim_employees)
df_dim_employees.head(2)

,employee_key,EmployeeID
0,1,1
1,2,2


In [48]:
# dim vendors

sql_dim_vendors = "SELECT vendor_key, VendorID FROM adventureworks_dw.dim_vendors;"
df_dim_vendors = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_dim_vendors)
df_dim_vendors.head(2)

,vendor_key,VendorID
0,1,1
1,2,2


In [49]:
# dim shipmethod

sql_dim_shipmethod = "SELECT shipmethod_key, ShipMethodID FROM adventureworks_dw.dim_shipmethod;"
df_dim_shipmethod = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_dim_shipmethod)
df_dim_shipmethod.head(2)

,shipmethod_key,ShipMethodID
0,1,1
1,2,2


### lookup surrogate primary keys 

In [50]:
# merge df_fact_po and dim_employees
# get 'employee_key' and drop 'EmployeeId' column 

df_fact_po = pd.merge(df_fact_po, df_dim_employees, on='EmployeeID', how='left')
df_fact_po.drop(columns=['EmployeeID'],inplace=True)
df_fact_po.head(2)

,PurchaseOrderID,RevisionNumber,Status,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,employee_key
0,1,0,4,83,3,2001-05-17,2001-05-26,201.0400,16.0832,5.0260,222.1492,2001-05-26,244
1,2,0,1,32,5,2001-05-17,2001-05-26,272.1015,21.7681,6.8025,300.6721,2001-05-26,231


In [51]:
# merge df_fact_po and dim_vendors
# get 'vendor_key' and drop 'VendorId' column 

df_fact_po = pd.merge(df_fact_po, df_dim_vendors, on='VendorID', how='left')
df_fact_po.drop(columns=['VendorID'],inplace=True)
df_fact_po.head(2)

,PurchaseOrderID,RevisionNumber,Status,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,employee_key,vendor_key
0,1,0,4,3,2001-05-17,2001-05-26,201.0400,16.0832,5.0260,222.1492,2001-05-26,244,83
1,2,0,1,5,2001-05-17,2001-05-26,272.1015,21.7681,6.8025,300.6721,2001-05-26,231,32


In [52]:
# merge df_fact_po and dim_shipmethod
# get 'shipmethod_key' and drop 'ShipMethodId' column 

df_fact_po = pd.merge(df_fact_po, df_dim_shipmethod, on='ShipMethodID', how='left')
df_fact_po.drop(columns=['ShipMethodID'],inplace=True)
df_fact_po.head(2)

,PurchaseOrderID,RevisionNumber,Status,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,employee_key,vendor_key,shipmethod_key
0,1,0,4,2001-05-17,2001-05-26,201.0400,16.0832,5.0260,222.1492,2001-05-26,244,83,3
1,2,0,1,2001-05-17,2001-05-26,272.1015,21.7681,6.8025,300.6721,2001-05-26,231,32,5


### lookup date keys 

In [53]:
# order date 

df_dim_order_date = df_dim_date.rename(columns={"date_key" : "order_date_key", "full_date" : "OrderDate"})
df_fact_po.OrderDate = df_fact_po.OrderDate.astype('datetime64[ns]').dt.date
df_fact_po = pd.merge(df_fact_po, df_dim_order_date, on='OrderDate', how='left')
df_fact_po.drop(columns=['OrderDate'], inplace=True)
df_fact_po.head(2)

,PurchaseOrderID,RevisionNumber,Status,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,employee_key,vendor_key,shipmethod_key,order_date_key
0,1,0,4,2001-05-26,201.0400,16.0832,5.0260,222.1492,2001-05-26,244,83,3,20010517
1,2,0,1,2001-05-26,272.1015,21.7681,6.8025,300.6721,2001-05-26,231,32,5,20010517


In [54]:
# ship date 

df_dim_ship_date = df_dim_date.rename(columns={"date_key" : "ship_date_key", "full_date" : "ShipDate"})
df_fact_po.ShipDate = df_fact_po.ShipDate.astype('datetime64[ns]').dt.date
df_fact_po = pd.merge(df_fact_po, df_dim_ship_date, on='ShipDate', how='left')
df_fact_po.drop(columns=['ShipDate'], inplace=True)
df_fact_po.head(2)

,PurchaseOrderID,RevisionNumber,Status,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,employee_key,vendor_key,shipmethod_key,order_date_key,ship_date_key
0,1,0,4,201.0400,16.0832,5.0260,222.1492,2001-05-26,244,83,3,20010517,20010526
1,2,0,1,272.1015,21.7681,6.8025,300.6721,2001-05-26,231,32,5,20010517,20010526


In [55]:
# modified date

df_dim_modified_date = df_dim_date.rename(columns={"date_key" : "modified_date_key", "full_date" : "ModifiedDate"})
df_fact_po.ModifiedDate = df_fact_po.ModifiedDate.astype('datetime64[ns]').dt.date
df_fact_po = pd.merge(df_fact_po, df_dim_modified_date, on='ModifiedDate', how='left')
df_fact_po.drop(columns=['ModifiedDate'], inplace=True)
df_fact_po.head(2)

,PurchaseOrderID,RevisionNumber,Status,SubTotal,TaxAmt,Freight,TotalDue,employee_key,vendor_key,shipmethod_key,order_date_key,ship_date_key,modified_date_key
0,1,0,4,201.0400,16.0832,5.0260,222.1492,244,83,3,20010517,20010526,20010526
1,2,0,1,272.1015,21.7681,6.8025,300.6721,231,32,5,20010517,20010526,20010526


### final transformations 

In [56]:
# reorder columns 

order_col = ['PurchaseOrderID','employee_key','vendor_key','shipmethod_key','order_date_key','ship_date_key','SubTotal',
             'TaxAmt','Freight','TotalDue','RevisionNumber','Status','modified_date_key']
df_fact_po = df_fact_po[order_col]

In [57]:
# insert primary key 

df_fact_po.insert(0, "purchase_order_key", range(1, df_fact_po.shape[0]+1))

In [58]:
# check work

df_fact_po.head(5)

,purchase_order_key,PurchaseOrderID,employee_key,vendor_key,shipmethod_key,order_date_key,ship_date_key,SubTotal,TaxAmt,Freight,TotalDue,RevisionNumber,Status,modified_date_key
0,1,1,244,83,3,20010517,20010526,201.0400,16.0832,5.0260,222.1492,0,4,20010526
1,2,2,231,32,5,20010517,20010526,272.1015,21.7681,6.8025,300.6721,0,1,20010526
2,3,3,241,38,2,20010517,20010526,8847.3000,707.7840,221.1825,9776.2665,0,4,20010526
3,4,4,266,85,5,20010517,20010526,171.0765,13.6861,4.2769,189.0395,0,3,20010526
4,5,5,164,92,4,20010531,20010609,20397.3000,1631.7840,509.9325,22539.0165,0,4,20010609


### push to SQL

In [59]:
# push to SQl

table_name = "fact_purchase_orders"
primary_key = "purchase_order_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_fact_po, table_name, primary_key, db_operation)

## Part 5: queries

In [64]:
# query to output the average total due of a purchase order for each vendor

sql_purchase_orders = """SELECT v.Name as vendor, AVG(po.TotalDue) as average_total_due
FROM adventureworks_dw.fact_purchase_orders as po
LEFT OUTER JOIN adventureworks_dw.dim_vendors as v
ON po.vendor_key = v.vendor_key
GROUP BY v.Name;
    
"""

In [65]:
df_query_1 = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_purchase_orders)
df_query_1

,vendor,average_total_due
0,"Litware, Inc.",167.712239
1,Advanced Bicycles,558.864545
2,Allenson Cycles,9776.266500
3,American Bicycles and Wheels,189.039500
4,American Bikes,22539.016500
...,...,...
74,"Vision Cycles, Inc.",55553.698220
75,Vista Road Bikes,41817.150400
76,West Junction Cycles,28212.058900
77,WestAmerica Bicycle Co.,501.200858


In [66]:
# query to output the overall subtotal for each employee

sql_employee_totals = """SELECT concat(e.FirstName, ' ', e.LastName) as employee, Sum(po.SubTotal) as overall_subtotal
FROM adventureworks_dw.fact_purchase_orders as po
LEFT OUTER JOIN adventureworks_dw.dim_employees as e
ON po.employee_key = e.employee_key
GROUP BY employee;
    
"""

In [67]:
df_query_2 = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_employee_totals)
df_query_2

,employee,overall_subtotal
0,Erin Hagens,4.633367e+06
1,Fukiko Ogisu,4.906001e+06
2,Eric Kurjan,6.283091e+06
3,Reinout Hillmann,6.536732e+06
4,Mikael Sandberg,6.169401e+06
5,Linda Meisner,6.718019e+06
6,Gordon Hee,5.705987e+06
7,Frank Pellow,5.908215e+06
8,Ben Miller,4.693242e+06
9,Sheela Word,2.263903e+06


In [76]:
# query to select all purchase orders between specific dates, outputting avg freight cost by shipmethod

sql_avg_freight_2002 = """SELECT s.Name as ship_method, AVG(po.Freight) as average_freight_cost
FROM adventureworks_dw.fact_purchase_orders as po
LEFT OUTER JOIN adventureworks_dw.dim_shipmethod as s
ON po.shipmethod_key = s.shipmethod_key
WHERE po.ship_date_key BETWEEN 20020101 AND 20021231 
GROUP BY s.Name;
    
"""

In [77]:
df_query_3 = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_avg_freight_2002)
df_query_3

,ship_method,average_freight_cost
0,CARGO TRANSPORT 5,475.224583
1,OVERNIGHT J-FAST,246.043712
2,ZY - EXPRESS,555.085007
3,XRQ - TRUCK GROUND,128.327267
4,OVERSEAS - DELUXE,819.872400


In [78]:
sql_avg_freight_2003 = """SELECT s.Name as ship_method, AVG(po.Freight) as average_freight_cost
FROM adventureworks_dw.fact_purchase_orders as po
LEFT OUTER JOIN adventureworks_dw.dim_shipmethod as s
ON po.shipmethod_key = s.shipmethod_key
WHERE po.ship_date_key BETWEEN 20030101 AND 20031231 
GROUP BY s.Name;
    
"""

In [79]:
df_query_3 = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_hostname, dst_dbname, sql_avg_freight_2003)
df_query_3

,ship_method,average_freight_cost
0,CARGO TRANSPORT 5,469.765403
1,ZY - EXPRESS,500.997360
2,XRQ - TRUCK GROUND,139.903619
3,OVERNIGHT J-FAST,255.923786
4,OVERSEAS - DELUXE,859.027160


From these queries, we can see that on average, cargo transport and express freight cost more in 2002 than in 2003, but average freight cost more in 2003 for express, overnight, and overseas transport.